In [1]:
### Install extra packages
!pip install sentinelsat

In [2]:
### Import modules
import sys
### Add path to scripts
if '/home/jovyan/leading_to_phytoplankton/scripts' not in sys.path:
    sys.path.insert(0, '/home/jovyan/leading_to_phytoplankton/scripts')
    

import numpy as np
from sentinelsat import SentinelAPI
import shapely.geometry as sg
from astropy.time import Time, TimeDelta
from tqdm import tqdm
import matplotlib.pyplot as plt
import h5py, pyproj
import readers as rd
from icepyx import icesat2data as ipd
import cartopy.crs as ccrs
import importlib
import search_sentinels as ss
import rasterio
from rasterio.plot import show
import pandas as pd
from osgeo import gdal

import warnings
warnings.filterwarnings('ignore')

### Magic function to enable interactive plotting
%matplotlib widget 

In [3]:
### Define path to sample data
data_loc = "/home/jovyan/shared/leading_to_phytoplankton/"
argo_arc_fn = data_loc+'argo_arc.csv'
argo_ant_fn = data_loc+'argo_ant.csv'

# Quick check of csv file contents
!head $argo_arc_fn

NaN,NaN,2018,6,20,17,45,4.33,0.00079445,28.762,34.726,0.2117
NaN,NaN,2018,6,20,17,45,5.6,0.00081808,28.764,34.726,0.2117
NaN,NaN,2018,6,20,17,45,11.36,0.00081808,28.771,34.726,0.2117
NaN,NaN,2018,6,20,17,45,16.69,0.00095985,28.773,34.726,0.2263
NaN,NaN,2018,6,20,17,45,21.73,0.00084171,28.774,34.727,0.2044
NaN,NaN,2018,6,20,17,45,26.77,0.00086534,28.774,34.727,0.219
NaN,NaN,2018,6,20,17,45,31.73,0.00085352,28.744,34.728,0.2555
NaN,NaN,2018,6,20,17,45,36.28,0.00087715,28.715,34.729,0.3139
NaN,NaN,2018,6,20,17,45,41.36,0.00086534,28.702,34.727,0.3504
NaN,NaN,2018,6,20,17,45,46.71,0.00084171,28.698,34.731,0.3942


In [4]:
!head $argo_ant_fn

-50.01,131.59,2018,9,14,11,17,0.44,-0.00043419,7.505,34.27,-0.00013273
-50.01,131.59,2018,9,14,11,17,1.54,-0.00042907,7.5033,34.267,-0.0035173
-50.01,131.59,2018,9,14,11,17,2.6,-0.0004344,7.4997,34.271,0
-50.01,131.59,2018,9,14,11,17,3.6,-0.00042339,7.493,34.278,0
-50.01,131.59,2018,9,14,11,17,4.5,-0.00042338,7.495,34.276,-0.0073
-50.01,131.59,2018,9,14,11,17,5.5,-0.00042338,7.4962,34.275,0
-50.01,131.59,2018,9,14,11,17,6.5,-0.00042338,7.497,34.276,0
-50.01,131.59,2018,9,14,11,17,7.4,-0.00043442,7.498,34.278,0
-50.01,131.59,2018,9,14,11,17,8.54,-0.00043393,7.4952,34.277,-0.0069756
-50.01,131.59,2018,9,14,11,17,9.54,-0.00042371,7.5001,34.277,-0.000146


In [5]:
### Define bounding box and time frame
bbox = [-110, 70, 70, 90]

!cat /home/jovyan/leading_to_phytoplankton/scripts/download.py

def download(bbox, timeframe, beam, earthdata_uid, email): 
    
    import os
    from icepyx import icesat2data as ipd
    from icepyx import core
    import readers as rd
    
    short_name = ATL03 '''fill in with name of whichever DataSet this is a member function of'''
    
    region.ipd.Icesat2Data(short_name, bbox, timeframe)
    region.avail_granules()
    
    print(region.avail_granules())
    if ('y' or 'yes' == input("Here's what we've got! Continue? (y/n)")):
        region.earthdata_login()
        
        path = './download/' + short_name
        region.download_granules(path)
        files = region.avail_granules(ids=True)
        
        print("Downloaded! Have a great day!")
    else:
        print("Nothing was downloaded")
        exit()

    
    for file in files:
        f = h5py.File(path+file, 'r')
        df = rd.getATL03(f,beam)
        
        #trim to bounding box
        df_cut=df[bbox]
        
        #convert time to UTC
        epoch=f['/ancillary_

In [6]:
### Load IS2 sample data
#f_atl03 = data_loc + "IS2_S2/ATL03_20190805215948_05930404_002_02.h5"
f_atl03 = data_loc + "IS2_S2/ATL03_20190726213326_04400404_002_01.h5"

f = h5py.File(f_atl03, 'r')
# check to see if it is forward (1)  or backward (0) orientation to know which beam is strong/weak. 
# (2 is transition phase- don't use these data)
print(f['orbit_info/sc_orient'][0])

beam = 'gt2l'
df03_full = rd.getATL03(f,beam)
print(df03_full.head())

print("Number of point measurements:", len(df03_full))

0
        lats       lons          x          y     heights            dt  conf
0  80.004734  54.366124  8938896.0  42.220333   89.935242  4.941201e+07     0
1  80.004744  54.366109  8938897.0  42.252739 -141.238495  4.941201e+07     0
2  80.004736  54.366121  8938897.0  42.207047  168.690918  4.941201e+07     0
3  80.004743  54.366112  8938897.0  42.228359   16.640863  4.941201e+07     0
4  80.004745  54.366109  8938898.0  42.234760  -29.026579  4.941201e+07     0
Number of point measurements: 10730607


In [7]:
### Downsample ATL03 to speed up test
df03 = df03_full[::1000]
print("Number of point measurements:", len(df03))

Number of point measurements: 10731


In [8]:
### Convert GPS time to UTC time
epoch = f['/ancillary_data/atlas_sdp_gps_epoch'][0]
df03['time'] = Time(epoch + df03['dt'],format='gps').utc.datetime

### Calculate along track distance relative to the beginning of the cut segment
df03['AT_dist'] = df03.x - df03.x.values[0]

### Read orbit number
df03['orbit_number'] = [f['/orbit_info/orbit_number'][0]] * len(df03)
df03.head()

,lats,lons,x,y,heights,dt,conf,time,AT_dist,orbit_number
0,80.004734,54.366124,8938896.0,42.220333,89.935242,4.941201e+07,0,2019-07-26 21:33:25.986627,0.0,4802
1000,80.007749,54.362400,8939241.0,41.681538,12.426127,4.941201e+07,0,2019-07-26 21:33:26.035227,345.0,4802
2000,80.008849,54.361042,8939366.0,41.479172,546.315491,4.941201e+07,0,2019-07-26 21:33:26.053327,470.0,4802
3000,80.009568,54.360142,8939448.0,41.564560,750.957275,4.941201e+07,0,2019-07-26 21:33:26.065027,552.0,4802
4000,80.010348,54.359147,8939537.0,42.044792,439.323090,4.941201e+07,0,2019-07-26 21:33:26.077426,641.0,4802


In [9]:
### Plot data
var = 'heights' #choose which variable we want to plot
vmin = -10
vmax = 30
ticks = np.arange(-20,100,5)

plt.figure(figsize=(8,8), dpi= 90)
ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=0)) # choose polar sterographic for projection
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.set_extent([-180, 180, 60, 90], ccrs.PlateCarree())
# plt.scatter(df03['lons'][::1000], df03['lats'][::1000],c=df03[var][::1000], cmap='viridis', vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
plt.scatter(df03['lons'], df03['lats'], c=df03[var], s=1, cmap='viridis', vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
plt.colorbar(label=var, shrink=0.5, ticks=ticks,extend='both')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
importlib.reload(ss)

### Retrieve collocated satellite imagery
ss.search_sentinels?

Signature:
ss.search_sentinels(
    platform_name,
    df,
    aoi,
    dt=2,
    user=None,
    pwd=None,
    proj_string='+init=EPSG:3995',
    product_type=None,
    min_cloud_cover=0,
    max_cloud_cover=100,
    swath_type=None,
    f_out=None,
)
Docstring:
Search Sentinel-1/2 images overlapping ICESat-2 data within +- dt

Parameters:
-----------
platform_name : str ['Sentinel-1 | Sentinel-2']
    name of the platform for which images will be searched
df : panda dataframe
    ICESat-2 data
aoi: str, list
    area of interest as WKT string or bounding box[lllon, lllat, urlon, urlat]
dt: int, float
    difference in hours between CS2 and S2
user : str
    username to connect to the Copernicus Scientific Hub
pwd : str
    password to connect to the Copernicus Scientific Hub
proj_string: str
    projection string to be used with the pyproj module
product_type : str
    name of the type of product to be searched (more info at https://scihub.copernicus.eu/userguide/)
swath_type : str
  

In [11]:
### Query Sentinel-1
#out_s1 = ss.search_sentinels('Sentinel-1', df03, bbox, dt=2, user='adia', pwd='ImageSearchTool2020')

In [12]:
### Query Sentinel-2
#out_s2 = ss.search_sentinels('Sentinel-2', df03, bbox, dt=1, user='adia', pwd='ImageSearchTool2020')

In [14]:
#out_s2


## Visualization

In [15]:
# Load the csv file with Pandas
argo_arc_df = pd.read_csv(argo_arc_fn)
argo_ant_df = pd.read_csv(argo_ant_fn)
# Add column names defined in the metadata
argo_arc_df.columns = ['lat', 'lon', 'year', 'month', 'day', 'hour', 
                   'minute', 'Depth (m)', 'bbp (700 nm)', 'temperature', 
                   'salinity', 'chlorophyll']
argo_ant_df.columns = ['lat', 'lon', 'year', 'month', 'day', 'hour', 
                   'minute', 'Depth (m)', 'bbp (700 nm)', 'temperature', 
                   'salinity', 'chlorophyll']


In [16]:
# Create a scatter plot showing data locations
plt.figure(figsize=(8,8), dpi= 90)
ax = plt.axes(projection=ccrs.PlateCarree()) # choose polar sterographic for projection
ax.coastlines(resolution='50m', color='black', linewidth=1)
plt.scatter(argo_arc_df['lon'], argo_arc_df['lat'], c= 'r',s=1,transform=ccrs.PlateCarree())
plt.scatter(argo_ant_df['lon'], argo_ant_df['lat'], c= 'r',s=1,transform=ccrs.PlateCarree())
ax.set_title('Location of Argo Floats');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
#s2_fn = 'IS2_S2/2019085_T11XMK.tif'
s2_fn = 'IS2_S2/20190726_T14XMQ.tif'
s2_img = rasterio.open(data_loc+s2_fn)
s2_bounds  = s2_img.bounds

from shapely.geometry import box
geom = box(*s2_bounds)
print(geom.wkt)
print(s2_img.crs)

POLYGON ((509760 8890200, 509760 9000000, 399960 9000000, 399960 8890200, 509760 8890200))
EPSG:32614


In [31]:
var= 'heights' #choose which variable we want to plot

## we will want to set colorbar parameters based on the chosen variable
vmin=-10
vmax=30
ticks=np.arange(-20,100,5)
proj = pyproj.Proj('+init=' + str(s2_img.crs))

plt.figure(figsize=(8,8), dpi= 90)
ax = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=0)) # choose polar sterographic for projection
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.set_extent([-180, 180, 60, 90], ccrs.PlateCarree())
plt.scatter(argo_arc_df['lon'], argo_arc_df['lat'], c= 'r',s=2,transform=ccrs.PlateCarree())
plt.scatter(df03['lons'], df03['lats'],c=df03[var], cmap='viridis', vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
plt.plot(*geom.exterior.xy, color='black', linewidth=1,transform=ccrs.CRS.proj4_init(ccrs.UTM(14)))
plt.colorbar(label=var, shrink=0.5, ticks=ticks,extend='both');

SyntaxError: invalid syntax (<ipython-input-31-06b984ec91b8>, line 15)

In [30]:
type)

cartopy.crs.UTM

In [19]:
plt.figure(figsize=(8,8), dpi= 90)
show(s2_img)


x, y = proj(np.array(df03['lons']), np.array(df03['lats']))
plt.plot(x[-1500:], y[-1500:], '.r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
plt.figure(figsize=(8,8), dpi= 90)
show(s2_img)
x_zoom, y_zoom = proj(np.array(df03_full['lons'][-1500000:]), np.array(df03_full['lats'][-1500000:]))
plt.plot(x_zoom, y_zoom, '.r')

plt.xlim((425027.75573844597, 427825.9806155204))
plt.ylim((8970799.98349305, 8972896.74600312))
lonmin, latmin = proj(plt.xlim()[0], plt.ylim()[0], inverse=True)
lonmax, latmax = proj(plt.xlim()[1], plt.ylim()[1], inverse=True)
print("lonmin:", lonmin)
print("lonmax:", lonmax)
print("latmin:", latmin)
print("latmax:", latmax)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

lonmin: -103.19261283470591
lonmax: -103.04480921439075
latmin: 80.77501361105543
latmax: 80.79552718811355


In [21]:
from scipy.interpolate import interp1d

### Load IS2 sample data
#f_atl03 = data_loc + "IS2_S2/ATL03_20190805215948_05930404_002_02.h5"
f_atl07 = data_loc + "IS2_S2/ATL07-01_20190726211228_04400401_002_02.h5"

f = h5py.File(f_atl07, 'r')

beam = 'gt2l'
df07 = rd.getATL07(f,beam)
print(df07.head())

print("Number of point measurements:", len(df07))

# mean sea surface correction - interpolating time with corrections
mss_corr=interp1d(df07.dt.values,df07.mss.values)
# inverted barometer correction
ib_corr=interp1d(df07.dt.values,df07.ib.values)
# ocean tide correction
ocean_corr=interp1d(df07.dt.values,df07.ocean.values)
# long period equilibrium tide correction
lpe_corr=interp1d(df07.dt.values,df07.lpe.values)

# Apply corrections  at sampling rate of dt from atl03
df03_full['correction']=mss_corr(df03_full.dt.values)+ib_corr(df03_full.dt.values)+ocean_corr(df03_full.dt.values)+lpe_corr(df03_full.dt.values)
df03_full['height_corr']=df03_full.heights-df03_full.correction # subtract correction from ATL03 heights
df03_full.head()

        lats       lons       heights            dt          conf  stype  \
0  79.560703  54.895848  3.402823e+38  4.941200e+07  3.402823e+38      1   
1  79.594908  54.856542  3.402823e+38  4.941200e+07  3.402823e+38      1   
2  79.643872  54.799941  3.402823e+38  4.941200e+07  3.402823e+38      1   
3  79.659694  54.781501  3.402823e+38  4.941200e+07  3.402823e+38      1   
4  79.674302  54.764400  3.402823e+38  4.941200e+07  3.402823e+38      1   

   ssh_flag         gauss   photon_rate  cloud       mss     ocean       lpe  \
0         0  3.402823e+38  3.402823e+38      5  8.248694 -0.052216  0.004915   
1         0  3.402823e+38  3.402823e+38      5  8.327850 -0.053121  0.004916   
2         0  3.402823e+38  3.402823e+38      5  8.446589 -0.054317  0.004919   
3         0  3.402823e+38  3.402823e+38      5  8.482800 -0.054679  0.004920   
4         0  3.402823e+38  3.402823e+38      5  8.515224 -0.055003  0.004920   

         ib  
0 -0.075599  
1 -0.075622  
2 -0.075685  
3 -0.0

,lats,lons,x,y,heights,dt,conf,correction,height_corr
0,80.004734,54.366124,8938896.0,42.220333,89.935242,4.941201e+07,0,6.933347e+37,-6.933347e+37
1,80.004744,54.366109,8938897.0,42.252739,-141.238495,4.941201e+07,0,6.933347e+37,-6.933347e+37
2,80.004736,54.366121,8938897.0,42.207047,168.690918,4.941201e+07,0,6.933942e+37,-6.933942e+37
3,80.004743,54.366112,8938897.0,42.228359,16.640863,4.941201e+07,0,6.933942e+37,-6.933942e+37
4,80.004745,54.366109,8938898.0,42.234760,-29.026579,4.941201e+07,0,6.933942e+37,-6.933942e+37


In [24]:
plt.figure()
plt.plot(df03_full['lats'], df03_full['height_corr'], ',r', label='ATL03')
plt.plot(df07['lats'], df07['heights'], '.k', label='ATL07')
plt.xlim((latmin, latmax))
plt.ylim((-7, 10))
plt.xlabel("lat [deg]")
plt.ylabel("h [m]")
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …